In [1]:
#titanic 예제에서의 BaseEstimator를 사용하여, 아무런 학습없이 성별만 가지고 예측해봅시당

from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음.
    def fit(self, X, y=None):
        pass
    #predict() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X["Sex"].iloc[1] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred

In [8]:
# Null 처리 함수
def fillna(df):
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    df["Cabin"].fillna("N", inplace=True)
    df["Embarked"].fillna("N", inplace=True)
    df["Fare"].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)
    return df

#레이블 인코딩 수행
def format_features(df):
    from sklearn import preprocessing
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할.
titanic_df = pd.read_csv("titanic_train.csv")
y_titanic_df = titanic_df["Survived"]
X_titanic_df = titanic_df.drop("Survived", axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

#DummyClassifier 로 한번 해봅시다
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print("Dummy Classifier의 정확도는: {0:.4f}".format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.6145


In [ ]:
#위의 결과를 볼때 어느정도 나쁘지 않은 결과가 나온다.

In [12]:
# MNIST 데이터 세트를 변환해, 불균형한 데이터 세트로 만든 뒤에 정확도 지표 적용시 어떤 문제가 발생하는지 봅시다.
# 숫자7을 찾아내는것 해봅시다. (binary classification)

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)
    
digits = load_digits()

# digits 번호가 7이면 True, astype(int) 1로 변환, 7이 아니면 False, 0으로 변환.
y = (digits.target==7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)




In [13]:
# 불균형한 레이블 데이터 분포도 확인.
print("레이블 테스트 세트 크기:", y_test.shape)
print("테스트 세트 레이블 0과 1의 분포도")
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print("모든 예측을 0으로 하여도 정확도는:{:.3f}".format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


In [ ]:
# 단지 모든값을 0으로만 예측해도 정확도가 90%가 된다??? 불균형한 레이블 데이터 세트는 성능 수치로 사용해서는 안된다;

In [ ]:
# 오차행렬 (이진 분류에서 성능지표로 잘 활용됨)

# 1. FP (예측값을 Positive 1로 예측하였으나 실제값은 0)
# 2. TN (예측값을 Negative 0으로 예측했고, 실제값 역시 0)
# 3. FN (예측값을 Negative 0으로 예측했고, 실제값은 1)
# 4. TP (예측값을 Positive 1로 예측, 실제값도 1)

In [14]:
# FP = 0, TN = 405, FN=45, TP=0
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)

# 정확도 = (TN+TP) / (TN+FP+FN+TP)

# 불균형한 이진 분류 데이터 세트에서는 Positive 데이터 건수가 매우 작기에, Negative로 예측 정확도가 높ㅍ아지는 경향이 발생.
# ex) 사기행위 = 1

array([[405,   0],
       [ 45,   0]])

In [ ]:
# 정밀도 = TP / (FP+TP) 
# - Positive로 예측한 대상중에 실제값이 positive로 일치한 데이터 비율

# 재현율(민감도) = TP / (FN+TP)
# - 실제값이 positive인 대상중에 예측과 실제 값이 positive로 일치한 데이터 비율

In [ ]:
# 재현율이 중요한 경우는, 실제 positive 양성 데이터를 negative로 잘못 판단하게 된 경우. (예를들어 암 판단 모델)
# 정밀도가 중요한경우는 (스팸메일 걸러내는것)